In [32]:
#Conversion from VGG to COCO annotations
import skimage 
import math
from itertools import chain
import numpy as np
import json
import os

def vgg_to_coco(dataset_dir, vgg_path, outfile, class_keyword):
    with open(vgg_path) as f:
        vgg = json.load(f)

    images_ids_dict = {}
    images_info = []
    for i,v in enumerate(vgg.values()):

        images_ids_dict[v["filename"]] = i
        image_path = os.path.join(dataset_dir, v['filename'])
        image = skimage.io.imread(image_path)
        height, width = image.shape[:2]  
        images_info.append({"file_name": v["filename"], "id": i, "width": width, "height": height})

    classes = {class_keyword} | {r["region_attributes"][class_keyword] for v in vgg.values() for r in v["regions"]
                             if class_keyword in r["region_attributes"]}
    category_ids_dict = {c: i for i, c in enumerate(classes, 1)}
    categories = [{"supercategory": class_keyword, "id": v, "name": k} for k, v in category_ids_dict.items()]
    annotations = []
    suffix_zeros = math.ceil(math.log10(len(vgg)))
    for i, v in enumerate(vgg.values()):
        #print(vgg.values())
        for j, r in enumerate(v["regions"]):
            print(r)
            if class_keyword in r["region_attributes"]:
                x, y = r["shape_attributes"]["all_points_x"], r["shape_attributes"]["all_points_y"]
                annotations.append({
                    "segmentation": [list(chain.from_iterable(zip(x, y)))],
                    "area": PolyArea(x, y),
                    "bbox": [min(x), min(y), max(x)-min(x), max(y)-min(y)],
                    "image_id": images_ids_dict[v["filename"]],
                    "category_id": category_ids_dict[r["region_attributes"][class_keyword]],
                    #"id": int(f'{i:0>{suffix_zeros}}{j:0>{suffix_zeros}}'),
                    "iscrowd": 0
                    })

    coco = {
        "images": images_info,
        "categories": categories,
        "annotations": annotations
        }
    if outfile is None:
        outfile = vgg_path.replace(".json", "_coco.json")
    with open(outfile, "w") as f:
        json.dump(coco, f)

In [33]:
vgg_path = '/home/martin/container/hail/Mask_RCNN/dataset/images/via_region_data_train.json'
dataset_dir = '/home/martin/container/hail/Mask_RCNN/dataset/images/train/'

vgg_to_coco(dataset_dir, vgg_path, None,"hail")

{'region_attributes': {'': '1'}, 'shape_attributes': {'all_points_x': [342.01, 339.61, 338.96, 338.47, 340.07, 340.9, 343.48, 346.38, 347.69, 348.48, 347.53, 348.17, 348.64, 348.82, 346.54, 344.6], 'name': 'polygon', 'all_points_y': [274.18, 276.12, 278.22, 280.32, 283.39, 284.36, 284.68, 285.17, 283.88, 282.26, 280.97, 279.35, 278.06, 275.47, 274.99, 274.02]}}
{'region_attributes': {'': '1'}, 'shape_attributes': {'all_points_x': [376.2, 374.6, 373.3, 372.8, 373.33, 375.3, 377.33, 379.5, 381.33, 384.13, 386.83, 389.33, 391.03, 392, 392.3, 391.73, 390.33, 388.43, 386.1, 383.3, 380.53, 378.4], 'name': 'polygon', 'all_points_y': [10.8, 12.7, 15.5, 19.5, 22.6, 24.9, 26.5, 27.3, 27.4, 27.5, 27.1, 24.7, 22.4, 19.2, 16.5, 14.29, 11.7, 10.1, 9, 8.7, 9.2, 9.7]}}
{'region_attributes': {'': '1'}, 'shape_attributes': {'all_points_x': [76.2, 76.25, 77.65, 79.65, 80.55, 80.65, 80.85, 79.35, 77.35, 76.45], 'name': 'polygon', 'all_points_y': [346.2, 344.51, 344.01, 344.01, 345.21, 347.01, 348.61, 348.

In [77]:
data['1']['regions']

[{'region_attributes': {'': '1'},
  'shape_attributes': {'all_points_x': [342.01,
    339.61,
    338.96,
    338.47,
    340.07,
    340.9,
    343.48,
    346.38,
    347.69,
    348.48,
    347.53,
    348.17,
    348.64,
    348.82,
    346.54,
    344.6],
   'all_points_y': [274.18,
    276.12,
    278.22,
    280.32,
    283.39,
    284.36,
    284.68,
    285.17,
    283.88,
    282.26,
    280.97,
    279.35,
    278.06,
    275.47,
    274.99,
    274.02],
   'name': 'polygon'}},
 {'region_attributes': {'': '1'},
  'shape_attributes': {'all_points_x': [376.2,
    374.6,
    373.3,
    372.8,
    373.33,
    375.3,
    377.33,
    379.5,
    381.33,
    384.13,
    386.83,
    389.33,
    391.03,
    392,
    392.3,
    391.73,
    390.33,
    388.43,
    386.1,
    383.3,
    380.53,
    378.4],
   'all_points_y': [10.8,
    12.7,
    15.5,
    19.5,
    22.6,
    24.9,
    26.5,
    27.3,
    27.4,
    27.5,
    27.1,
    24.7,
    22.4,
    19.2,
    16.5,
    14.29,
    11.

In [39]:
import json
#Load example file to adjust with new polygon data
with open('/home/martin/container/hail/Mask_RCNN/dataset/images/via_region_data_train.json') as f:
    data = json.load(f)
    
#variables to update in json file
for file_id, filename in enumerate(all_train_files):
    if file_id > 0:
        data[str(file_id + 1)] = data['1']
        
    filename
all_points_x = []
all_points_y = []


In [83]:
data['1']

{'file_attributes': {'height': 500, 'width': 500},
 'filename': 'odm_orth_15_36.png',
 'regions': [{'region_attributes': {'': '1'},
   'shape_attributes': {'all_points_x': [342.01,
     339.61,
     338.96,
     338.47,
     340.07,
     340.9,
     343.48,
     346.38,
     347.69,
     348.48,
     347.53,
     348.17,
     348.64,
     348.82,
     346.54,
     344.6],
    'all_points_y': [274.18,
     276.12,
     278.22,
     280.32,
     283.39,
     284.36,
     284.68,
     285.17,
     283.88,
     282.26,
     280.97,
     279.35,
     278.06,
     275.47,
     274.99,
     274.02],
    'name': 'polygon'}},
  {'region_attributes': {'': '1'},
   'shape_attributes': {'all_points_x': [376.2,
     374.6,
     373.3,
     372.8,
     373.33,
     375.3,
     377.33,
     379.5,
     381.33,
     384.13,
     386.83,
     389.33,
     391.03,
     392,
     392.3,
     391.73,
     390.33,
     388.43,
     386.1,
     383.3,
     380.53,
     378.4],
    'all_points_y': [10.8,
   

In [17]:
data['1']['regions'][0]['shape_attributes']['all_points_y']

[274.18,
 276.12,
 278.22,
 280.32,
 283.39,
 284.36,
 284.68,
 285.17,
 283.88,
 282.26,
 280.97,
 279.35,
 278.06,
 275.47,
 274.99,
 274.02]